In [1]:
import pandas as pd

data = pd.read_csv('ratings3.csv')
data.index = ['John' , 'Mary', 'Lee' , 'Joe' , 'Kim' , 'Bob']
print(data)

      Mission Impossible  Over the Hedge  Back to the Future  Harry Potter  \
John                 5.0               3                 4.0           NaN   
Mary                 5.0               4                 5.0           5.0   
Lee                  2.0               2                 4.0           5.0   
Joe                  3.0               1                 1.0           2.0   
Kim                  4.0               5                 NaN           2.0   
Bob                  NaN               1                 5.0           4.0   

      Aladdin  
John        3  
Mary        5  
Lee         1  
Joe         1  
Kim         2  
Bob         2  


In [5]:
import numpy as np

def is_number(num):
    return num == num and type(num) != str



def baseline(matrix, lambda1, lambda2, max_iter):
    
    # Initialize the model parameters w_u, w_i, and mu
    w_u = np.zeros(np.size(matrix,1))
    w_i = np.zeros(np.size(matrix,1))
    clean_matrix =  matrix[~np.isnan(matrix)]
    mu =  clean_matrix.mean()
    
    for z in range(max_iter):
        # Update the model parameters using the formula shown in class
        
        #calculate w_u
        temp = [0,0,0,0,0,0]
        for i,row in enumerate(matrix):
           # print("row: " , row)
            rating_count = 0
            for j, item in enumerate(row):
                if is_number(item):
                    rating_count += 1
                    temp[i] += item - w_i[j] - mu
            temp[i] = temp[i]/(rating_count - lambda1)
        w_u = temp
    
        #calculate w_i
        temp = [0,0,0,0,0]
        data_t = data.T   
        for i , column in enumerate(data_t.iterrows()):
            rating_count = 0
            for j, entry in enumerate(column):   
                    for rating in entry:
                        if is_number(rating):
                            rating_count += 1
                            temp[i] += rating - w_i[j] - mu
            temp[i] = temp[i] / (rating_count - lambda2)
        w_i = temp
                        
        #calculate mu again
        temp = 0
        rating_count = 0
        for i,row in enumerate(matrix):
            for j, item in enumerate(row):
                if is_number(item):
                    temp += item - w_u[i] - w_i[j]
                    rating_count += 1
    
        mu = temp/rating_count
    
        #Display the updated model parameters
        print('Iteration ', z)
        print('   w_u =', np.around(w_u , 3))
        print('   w_i =', np.around(w_i , 3))
        print('   mu  =', round(mu , 4))

    return (w_u, w_i, mu)

In [6]:
w_u, w_i, mu = baseline(data.values, 0, 0, 5)


print('Predicted rating for John on Harry Potter =',  round(w_u[1] + w_i[3] + mu , 3))
print('Predicted rating for Kim on Back to the Future =', round( w_u[4] + w_i[2] + mu , 3))
print('Predicted rating for Bob on Mission Impossible =',  round(w_u[5] + w_i[0] + mu , 3))


Iteration  0
   w_u = [ 0.565  1.615 -0.385 -1.585  0.065 -0.185]
   w_i = [ 0.615 -0.519  0.615  0.415 -0.852]
   mu  = 3.1852
Iteration  1
   w_u = [ 0.6   1.56 -0.44 -1.64  0.15 -0.1 ]
   w_i = [ 1.133  0.     1.133  0.933 -0.333]
   mu  = 2.6667
Iteration  2
   w_u = [ 0.6   1.56 -0.44 -1.64  0.15 -0.1 ]
   w_i = [ 1.133  0.     1.133  0.933 -0.333]
   mu  = 2.6667
Iteration  3
   w_u = [ 0.6   1.56 -0.44 -1.64  0.15 -0.1 ]
   w_i = [ 1.133  0.     1.133  0.933 -0.333]
   mu  = 2.6667
Iteration  4
   w_u = [ 0.6   1.56 -0.44 -1.64  0.15 -0.1 ]
   w_i = [ 1.133 -0.     1.133  0.933 -0.333]
   mu  = 2.6667
Predicted rating for John on Harry Potter = 5.16
Predicted rating for Kim on Back to the Future = 3.95
Predicted rating for Bob on Mission Impossible = 3.7


In [9]:
def MF(matrix, k, maxiter):
    
    # Initialize the missing ratings in matrix to 0
    matrix[np.isnan(matrix)] = 0

   # Initialize the matrices U and M to 1s. Size of U is #rows x k, size of M is $columns x k
    U = np.full((matrix.shape[0] , k) , 1).astype(float)
    M = np.full((matrix.shape[1] , k) , 1).astype(float)


    for i in range(maxiter):
        
        # Update M
        for i , row in enumerate(M):
            for j , element in enumerate(row):
                M[i][j] = M[i][j] *( (matrix.T.dot(U)[i][j])  / (M.dot(U.T).dot(U)[i][j]) )

        
    
        # Update U
        for i , row in enumerate(U):
            for j , element in enumerate(row):
                U[i][j] = U[i][j] * (matrix.dot(M)[i][j]) / ( (U.dot(M.T).dot(M))[i][j]) 
        
        # Update the missing ratings in matrix with the predicted values U x M^T 
        
        
        
    return U, M

In [10]:
U, M = MF(data.values, 2, 1)
predicted =  U.dot(M.T)

print('Predicted rating for John on Harry Potter =', predicted[0][3])
print('Predicted rating for Kim on Back to the Future =', predicted[4][2])
print('Predicted rating for Bob on Mission Impossible =', predicted[5][0])


Predicted rating for John on Harry Potter = 3.003169567202251
Predicted rating for Kim on Back to the Future = 2.7675115107776556
Predicted rating for Bob on Mission Impossible = 2.654167825200697


In [11]:
"""
3
    A)  S.Name 
        Bart Simpson 
        
        
    B)  S.Name            COUNT(T.CrsCode)
        Joseph Public     3
        Bart Simpson      2
        Joe Blow          3
        Homer Simpson     3
        John Doe          3
        
        
    C)  C.CrsName         T1.Semester
        Market Analysis   F1994
        Market Analysis   F1997
      
      
    D)  S.Name 
        Bart Simpson
        Homer Simpson
        Joe Blow



4
    A) 
        SELECT PH.Name
        FROM Physician PH, Patient PA, Visit V
        WHERE PA.Name = 'John Doe' AND
              V.PatientID = PA.ID AND
              V.PhysicianID = PH.ID;
              
              
   B) 
        SELECT DISTINCT P.ID, P.Name
        FROM Insurance I, Visit V, Patient P
        WHERE V.PatientID = I.PatientID AND
              I.Company IS NOT NULL AND
              P.IID = V.PatientID;
              
              
   C) 
        SELECT DISTINCT P.ID, P.Name
        FROM Insurance I, Visit V, Patient P
        WHERE V.PatientID = I.PatientID AND
              I.Company IS NULL AND
              P.ID = V.PatientID;
              
   D) 
        SELECT DISTINCT P.ID, P.Name
        FROM Visit V, Patient P
        WHERE V.PatientID = P.ID
        HAVING COUNT(Distinct V.PhysicianID) > 1;
              
   E) 
        SELECT DISTINCT P.ID, P.Name, COUNT(*)
        FROM Physician P, Visit V
        WHERE V.PhysicianID = P.ID;
        
        
        
        
5
   A)
        Attributes: Customer.ID, Customer.Name, C.Address,
                    Rental.CustomerID, Rental.BikeID, Rental.CheckoutDate, Rental.ReturnDate
            
        Number of rows: Minimum = 0
                        Maximum = 11,000
                        
                        
                        
   B)
        Attributes: Customer.Name, Bike.Type
        
        Number of rows: Minimum = 0
                        Maximum = 10,000
                        
   C)
        Attributes: Bike.ID, Bike.Type            

        Number of rows: Minimum = 0
                        Maximum = 500
                                       
                                       
   C)
        Attributes: Bike.ID, Bike.Type
            
        Number of rows: Minimum = 0
                        Maximum = 500


   D)
        Attributes: Customer.ID, Customer.Name, C.Address,
                    Bike.ID, Bike.Type
            
        Number of rows: Minimum = 0
                        Maximum = 1500
                        
                        
   E)
        Attributes: Customer.ID, Bike.Type, Num
            
        Number of rows: Minimum = 0
                        Maximum = 10,000
"""

"\n3\n    A)  S.Name \n        Bart Simpson \n        \n        \n    B)  S.Name            COUNT(T.CrsCode)\n        Joseph Public     3\n        Bart Simpson      2\n        Joe Blow          3\n        Homer Simpson     3\n        John Doe          3\n        \n        \n    C)  C.CrsName         T1.Semester\n        Market Analysis   F1994\n        Market Analysis   F1997\n      \n      \n    D)  S.Name \n        Bart Simpson\n        Homer Simpson\n        Joe Blow\n\n\n4\n    A) \n        SELECT PH.Name\n        FROM Physician PH, Patient PA, Visit V\n        WHERE PA.Name = 'John Doe' AND\n              V.PatientID = PA.ID AND\n              V.PhysicianID = PH.ID;\n              \n              \n   B) \n        SELECT DISTINCT P.ID, P.Name\n        FROM Insurance I, Visit V, Patient P\n        WHERE V.PatientID = I.PatientID AND\n              I.Company IS NOT NULL AND\n              P.IID = V.PatientID;\n              \n              \n   C) \n        SELECT DISTINCT P.ID, P.N